In [208]:
import numpy as np
import keras
from keras.models import Sequential,Model
from keras.layers import Activation,MaxPooling2D,Dropout,Concatenate,Input,Add
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from tensorflow.keras.models import load_model
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping,LearningRateScheduler,CSVLogger,LambdaCallback,TensorBoard
from keras import regularizers
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.regularizers import l2
import seaborn as sns
import math
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/'
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.utils import plot_model
from sklearn.metrics import classification_report
%matplotlib inline

In [209]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
train_path = 'cats-and-dogs/train'
valid_path = 'cats-and-dogs/valid'
test_path  = 'cats-and-dogs/test'
save_path = "."
train_gen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    channel_shift_range=10.,
    fill_mode="nearest"
)

train_batches = train_gen.flow_from_directory(directory=train_path, target_size=(150,150),
    classes=['dog', 'cat'], batch_size=16)
valid_batches = ImageDataGenerator().flow_from_directory(directory=valid_path, target_size=(150,150), 
    classes=['dog', 'cat'], batch_size=16)
test_batches = ImageDataGenerator().flow_from_directory(directory=test_path, target_size=(150,150),
    classes=['dog', 'cat'], batch_size=16,shuffle=False)

Found 10514 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [210]:
lr_print_callback = LambdaCallback(
    on_epoch_begin=lambda epoch,logs: print("LearningRate of %e",model.optimizer.lr))

earlyStopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    verbose=0,
    mode='auto',
    baseline=None,
	patience=10,
    restore_best_weights=True
)

reduce_lr_loss = ReduceLROnPlateau(
    monitor='val_loss', 
	factor=0.1, 
    patience=7, 
	verbose=1, 
    min_delta=1e-4,
	mode='auto')


reduce_lr_loss = ReduceLROnPlateau(
    monitor='val_loss', 
	factor=0.1, 
    patience=7, 
	verbose=1, 
    min_delta=1e-4,
	mode='auto')

mcp_save       = ModelCheckpoint(
    'cat_dog_ckpt_densenet.h5', 
	save_best_only=True, 
	monitor='val_loss', 
	mode='auto'
)

In [211]:
def conv_block(input_,num_kernel,growth_rate,kernel_size):
	bn   =  BatchNormalization()(input_)
	act  = Activation('relu')(bn)
	conv = Conv2D(num_kernel, (3,3), padding='same',
                   kernel_initializer='glorot_uniform',
                   bias_initializer='zeros', kernel_regularizer=l2(0.0005))(act)
	return conv


In [212]:
'''Create a dense block which will connect a layer with all previous layer using concatenation,instead of add as in resnet'''
def dense_block(num_dense_blk,x,nb_channels,growth_rate,kernel_size):
    x_list = [x]
    for i in range(num_dense_blk):
        cb = conv_block(x,nb_channels, growth_rate,kernel_size)
        x_list.append(cb)
        x = Concatenate(axis=-1)(x_list)
    nb_channels += growth_rate
    return x, nb_channels

In [213]:
'''Creates a transition layer between dense blocks as transition, which do convolution and pooling.
    Transition block uses 1x1 conv2d for downsampling'''
def transition_block(x, nb_channels,growth_rate):
  
	x = BatchNormalization()(x)
	x = Activation('relu')(x)
	x = Conv2D(nb_channels, (1, 1), padding='same',
                      use_bias=False, kernel_regularizer=l2(0.0005))(x)
	x = MaxPooling2D((2, 2), strides=(2, 2))(x)
	nb_channels = nb_channels + growth_rate
	return x,nb_channels

In [221]:

def output_layer(x,num_kernel):
	x = BatchNormalization()(x)
	x = Activation('relu')(x)
	x = Conv2D(num_kernel, (3,3), padding='same',
               kernel_initializer='glorot_uniform',
               bias_initializer='zeros', kernel_regularizer=l2(0.0005))(x)
	x = MaxPooling2D((2, 2), strides=(2, 2))(x)
	x = Dropout(0.5)(x)
	x = Flatten()(x)
	x = Dense(2, activation='sigmoid')(x)

	return x


In [222]:
growth_rate =16
nb_channels = growth_rate * 2
visible    = Input(shape=(150, 150, 3))
kernel_size = [(3,3),(3,3),(3,3),(3,3)]
num_layers_dense_blk = 3
x = Conv2D(nb_channels, (3,3), padding='same',strides=(1,1),
                      use_bias=False, kernel_regularizer=l2(0.0005))(visible)
for i in range(4):
    x, nb_channels = dense_block(num_layers_dense_blk,x,nb_channels,growth_rate,kernel_size[i])
    x, nb_channels = transition_block(x, nb_channels,growth_rate)
x = output_layer(x,nb_channels)
model = Model(inputs=visible, outputs=x)

model.save('densenet_experiment.h5')
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.0001),metrics=['accuracy'])

In [ ]:

history = model.fit_generator(generator=train_batches, steps_per_epoch=len(train_batches), 
                              validation_data=valid_batches, validation_steps=len(valid_batches), 
                              epochs=200, callbacks=[reduce_lr_loss],verbose=1) 
acc = history.history['accuracy'] 
val_acc = history.history['val_accuracy'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.legend() 
plt.figure() 
plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss') 
plt.savefig('accuracy.png')
plt.show()
model.save('densenet_experiment.h5')

Epoch 1/200
658/658 [==============================] - 575s 874ms/step - loss: 1.6647 - accuracy: 0.6018 - val_loss: 1.6482 - val_accuracy: 0.6863
Epoch 2/200
658/658 [==============================] - 577s 877ms/step - loss: 1.5642 - accuracy: 0.6742 - val_loss: 2.5969 - val_accuracy: 0.6903
Epoch 3/200
658/658 [==============================] - 578s 879ms/step - loss: 1.4948 - accuracy: 0.7084 - val_loss: 1.2764 - val_accuracy: 0.7759
Epoch 4/200
658/658 [==============================] - 580s 881ms/step - loss: 1.4367 - accuracy: 0.7379 - val_loss: 1.1945 - val_accuracy: 0.7726
Epoch 5/200
658/658 [==============================] - 577s 876ms/step - loss: 1.3890 - accuracy: 0.7582 - val_loss: 1.2007 - val_accuracy: 0.8151
Epoch 6/200
658/658 [==============================] - 575s 874ms/step - loss: 1.3193 - accuracy: 0.7887 - val_loss: 1.1279 - val_accuracy: 0.7399
Epoch 7/200
658/658 [==============================] - 578s 878ms/step - loss: 1.2665 - accuracy: 0.8060 - val_loss: 1

In [ ]:
predictions = model.predict_generator(generator=test_batches, steps=len(test_batches), verbose=0)
pred_label = np.argmax(predictions,axis=1)
classes = np.argmax(predictions, axis=1)
cm = confusion_matrix(test_batches.labels,pred_label)
f,ax = plt.subplots(figsize=(4, 4))
sns.heatmap(cm, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()
plt.savefig('confusion_matrix.png')

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import metrics
print(classification_report(test_batches.classes, pred_label,
	target_names=test_batches.class_indices.keys()))
tn, fp, fn, tp = cm.ravel()
accuracy = (tn + tp)/(tn + tp + fn +fp)
precision = precision_score(test_batches.labels, pred_label, average='binary')
recall = recall_score(test_batches.labels, pred_label,average='binary')
f1_score = f1_score(test_batches.labels, pred_label, average='binary')
score = metrics.accuracy_score(test_batches.labels, pred_label)
log_score = metrics.log_loss(pred_label, predictions)
print("Precision ",precision*100)
print("Recall ",recall*100)
print("F1 Score ",recall*100)
print("Accuracy of the model",accuracy*100)
print("Accuracy score: {}".format(score))
print("Log loss score: {}".format(log_score))

In [ ]:
from sklearn.metrics import roc_curve, auc
def plot_roc(pred,y):
    fpr, tpr, _ = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()
plot_roc(pred_label,test_batches.labels)